In [1]:
import os
import collections
from collections import defaultdict
from tqdm import tqdm
import json
import pickle
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import seaborn as sns
import numpy as np
import pandas as pd
from langdetect import detect, LangDetectException

import matplotlib.pyplot as plt
from adjustText import adjust_text

import langid
from src.wimbd_ import WimbdAnalysis, load_mmlu, DataConfigs

[nltk_data] Downloading package punkt to
[nltk_data]     /local/home/antonis/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /local/home/antonis/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[dynet] random seed: 1234
[dynet] allocating memory: 32MB
[dynet] memory allocation done.


In [2]:
DataConfigs.mmlu_tasks['professional']

['professional_psychology', 'professional_law']

In [3]:
n_gram_freqs_path = "./results/n-grams/exp_full/2/common/pl-en-2-grams.pkl"
n_gram_freqs = pickle.load(open(n_gram_freqs_path, "rb"))
n_gram_freqs = dict(sorted(n_gram_freqs.items(), key=lambda item: item[1]['value'], reverse=True))
# # export as text
# # with open("n_gram_freqs.txt", "w") as f:
# #     for k, v in n_gr
# # 3am_freqs.items():
# #         f.write(f"{k}: {v}\n")
# LANGUAGES = ['ru-en', 'fr-en', 'ro-en', 'de-en', 'pl-en', 'cs-en']
# LANGUAGES =  ['ru-en', 'ro-en', 'de-en', 'pl-en', 'cs-en', 'fr-en', 'ja-en', 'zh-en']

# wmt09
TASKS = DataConfigs.mmlu_tasks['professional']
ds = load_mmlu(TASKS)

In [ ]:
# lang params
N_GRAMS = 5
# BASE_DIR = "./results/n-grams/mmlu/test-set/exp_full_None"
BASE_DIR = "./results/n-grams/mmlu/exp3/test-set/exp_full_None"
# BASE_DIR = "./results/n-grams/mmlu/pile/exp4_filter/test-set/exp_full_None"
# BASE_DIR = "./results/n-grams/mmlu/pile/exp4_nofilter/test-set/exp_full_None"
BASE_PATH = os.path.join(BASE_DIR, f"{N_GRAMS}")
BASE_PATH_COMMON = os.path.join(BASE_PATH, "common")
BASE_PATH_ALL = os.path.join(BASE_PATH, "all")
FILTER_CHARS = False
DETECT_LANG = False
LOG_AXIS = True
TASK = "MMLU/hendrycks*"
SHOT = "0-shot"

# model params
base_results_paths = {
    "/share/edc/home/antonis/LLM-Incidental-Supervision/incidental-supervision/models/pythia/experiment_4/inference/EleutherAI": [
        'pythia-12b', 'pythia-6.9b', 'pythia-2.8b', 
        'pythia-1.4b', 'pythia-410m', 'pythia-160m', 
        'pythia-70m', 'pythia-31m', 'pythia-14m'
    ],
    "/share/edc/home/antonis/LLM-Incidental-Supervision/incidental-supervision/models/OLMO": [
        'OLMo-7b'
    ]
}

models = ['pythia-12b', 'pythia-6.9b', 'pythia-2.8b', 
          'pythia-1.4b', 'pythia-410m', 'pythia-160m', 
          'pythia-70m', 'pythia-31m', 'pythia-14m',
          'OLMo-7b']

In [ ]:
TASKS_OMMIT = ["formal_logic"]
# TASKS = [t for t in list(ds.keys()) if t not in TASKS_OMMIT]
TASKS = MMLU_TASKS = DataConfigs.mmlu_tasks['professional']
wa = WimbdAnalysis(BASE_PATH, TASKS, N_GRAMS, FILTER_CHARS)

In [ ]:
task_dfs, task_dfs_path = wa.get_task_dfs(BASE_PATH_COMMON, TASKS, filter_chars=FILTER_CHARS)
task_dfs_filtered, task_dfs_filtered_path = wa.get_task_dfs(BASE_PATH, TASKS, filter_chars=True)
# task_dfs

# get total samples per language pair
task_dfs_total_samples = {lang: df['value'].sum() for lang, df in task_dfs.items()}
colors = sns.color_palette('hls', len(task_dfs))
color_mapping = {lang: color for lang, color in zip(task_dfs.keys(), colors)}

In [ ]:
# ds = {task: ds[task] for task in task_dfs.keys()}
# TASKS = [t for t in list(ds.keys()) if t not in TASKS_OMMIT]

In [ ]:
total_samples = 0
for task in TASKS:
    total_samples += task_dfs_total_samples[task]
print(f"Total samples: {total_samples}")

In [ ]:
task_dfs

In [ ]:
from src.analysis import AnalyzeNgrams as an

coverage_path_common = os.path.join(BASE_PATH_COMMON, "task-coverage.pkl")
coverage_path_all = os.path.join(BASE_PATH_ALL, "task-coverage.pkl")

task_cov_all, task_cov_all_mean = an.calculate_average_task_coverage(BASE_PATH_ALL, TASKS, [N_GRAMS])
task_cov_common, task_cov_common_mean = an.calculate_average_task_coverage(BASE_PATH_COMMON, TASKS, [N_GRAMS])

In [ ]:
task_cov

In [ ]:
task_dfs_all, task_dfs_all_path = wa.get_task_dfs(BASE_PATH_ALL, TASKS)
task_dfs_common, task_dfs_common_path = wa.get_task_dfs(BASE_PATH_COMMON, TASKS)

In [ ]:
# # export as .pkl
# # save all tasks .pkl

# with open(os.path.join(BASE_PATH, "task_dfs_all.pkl"), "wb") as f:
#     pickle.dump(task_dfs_all, f)

# with open(os.path.join(BASE_PATH, "task_dfs_common.pkl"), "wb") as f:
#     pickle.dump(task_dfs_common, f)

In [ ]:
# task_dfs_common['college_mathematics']
# task_dfs_common['college_mathematics']['example'].iloc[11]

In [ ]:
pth = "/share/edc/home/antonis/LLM-Incidental-Supervision/incidental-supervision/results/n-grams/mmlu/exp1/4/common/task-coverage.pkl"

with open(pth, "rb") as f:
    df = pickle.load(f)

pd.DataFrame(df)

In [ ]:
# save_path = f"./results/n-grams/exp_full/{N_GRAMS}/common"
# # for each language, save top 100 examples in a csv

# for lang, df in lang_dfs.items():
#     df = df.sort_values(by=['value'], ascending=False)
#     df.to_csv(os.path.join(save_path, f"{lang}_samples.csv"), index=False)

# for lang, df in lang_dfs_filtered.items():
#     df = df.sort_values(by=['value'], ascending=False)
#     df = df.head(100)
#     df.to_csv(os.path.join(save_path, f"{lang}_samples_filtered.csv"), index=False)


In [ ]:
# n_gram_freqs.pop(('angela merkel', 'chancellor angela'))
n_gram_freqs_values = [v['value'] for v in n_gram_freqs.values()]
n_gram_freqs_values_cumsum = np.cumsum(n_gram_freqs_values)


In [ ]:
# # Usage:
# wa.plot_n_samples(task_dfs, color_mapping)
# # wa.plot_n_samples(lang_dfs_filtered, color_mapping)

In [ ]:
model_colormap = plt.cm.get_cmap('coolwarm', len(models))
model_color_mapping = {model: model_colormap(1 - i / len(models)) for i, model in enumerate(models)}


# wa.analyze_and_plot_distributions(task_dfs)

In [ ]:
# Initialize a dictionary to hold the contents of results.json for each model/dataset pair
results_dict = collections.defaultdict(dict)
instance_results_dict = collections.defaultdict(dict)

# Iterate over each model and dataset, loading the results.json file

for base_results_path, models_ in base_results_paths.items():
    for model in models_:
        results_path = os.path.join(base_results_path, model, TASK, SHOT, 'results.json')
        instance_results_path = os.path.join(base_results_path, model, TASK, SHOT, 'doc_results.json')
        results = json.load(open(results_path, 'r'))['results']
        doc_results = json.load(open(instance_results_path, 'r'))
        for task in results.keys():
            # remove hendrycksTest-
            task_str = task.split("-")[1]
            if task_str in TASKS:
                results_dict[task_str][model] = results[task]['acc']
                if os.path.exists(instance_results_path):
                    instance_results_dict[model][task_str] = doc_results[task]

In [ ]:
wa.plot_bleu_scores(results_dict, models, color_mapping)

task_ds = ds

# task_mutual_info, merged_mutual_info = wa.calculate_mutual_info(task_dfs, task_ds)
task_mutual_info, merged_mutual_info = None, None

In [ ]:
task_dfs.keys()

In [ ]:
model_scores, dataset_scores = wa.prepare_scores(results_dict, task_dfs, 
                                                 models, coverage_=task_cov_common_mean,
                                                 cov_mean=task_cov_common_mean)

single_model = ["pythia-12b"]
single_model_score, single_model_dataset_score = wa.prepare_scores(results_dict, task_dfs, 
                                                                   single_model, coverage_=task_cov_common_mean)

In [ ]:
def plot_average_scores(dataset_scores, model_scores, color_mapping, 
                       model_list=None, x_key='n_samples',
                       log_axis=False, title=None, annotate=False):
    avg_dataset_scores = collections.defaultdict(lambda: collections.defaultdict(list))
    avg_scores = {'score': [], x_key: []} 

    for dataset in dataset_scores:
        if model_list is None:
            avg_dataset_scores[dataset]['score'] = np.mean(dataset_scores[dataset]['score'])
            avg_dataset_scores[dataset][x_key] = np.mean(dataset_scores[dataset][x_key])
        else:
            model_indices = [i for i, model in enumerate(model_scores.keys()) if model in model_list]
            avg_dataset_scores[dataset]['score'] = np.mean([dataset_scores[dataset]['score'][i] for i in model_indices])
            avg_dataset_scores[dataset][x_key] = np.mean([dataset_scores[dataset][x_key][i] for i in model_indices])

        plt.scatter(avg_dataset_scores[dataset][x_key], avg_dataset_scores[dataset]['score'], 
                    color=color_mapping[dataset], label=dataset)
        if annotate:
            plt.annotate(dataset, (avg_dataset_scores[dataset][x_key], avg_dataset_scores[dataset]['score']))
        avg_scores['score'].append(avg_dataset_scores[dataset]['score'])
        avg_scores[x_key].append(avg_dataset_scores[dataset][x_key])

    # sort by n_samples
    avg_scores['score'] = [x for _, x in sorted(zip(avg_scores[x_key], avg_scores['score']))]
    avg_scores[x_key] = sorted(avg_scores[x_key])
    
    plt.plot(avg_scores[x_key], avg_scores['score'], color='red', label='large models')
    plt.xlabel(x_key)
    plt.ylabel('score')
    if title is not None:
        plt.title(title)
    if log_axis:
        plt.xscale('log')

    if title is not None:    
        title_path = title.replace(' ', '_').replace(',', '_')
        if not os.path.exists(wa.plot_path):
            os.makedirs(wa.plot_path)
        plt.savefig(os.path.join(wa.plot_path, f"avg_scores_{title_path}.png"))
    
    plt.show()

    return avg_scores

large_models = ["pythia-12b", "pythia-6.9b", "pythia-2.8b", "pythia-1.4b"]
small_models = ["pythia-160m", "pythia-70m", "pythia-31m", "pythia-14m"]

# avg_scores = plot_average_scores(dataset_scores, model_scores, color_mapping)
avg_scores_large = plot_average_scores(dataset_scores, model_scores, 
                                      color_mapping, large_models,
                                      log_axis=LOG_AXIS,
                                      title="large_models, cov_mean, COMMON INSTANCES",)
avg_scores_small = plot_average_scores(dataset_scores, model_scores,
                                        color_mapping, small_models,
                                        log_axis=LOG_AXIS,
                                        x_key='cov_mean',
                                        title="small_models, cov_mean, COMMON INSTANCES",)
avg_scores = plot_average_scores(dataset_scores, model_scores,
                                        color_mapping, models,
                                        log_axis=LOG_AXIS,
                                        x_key='cov_mean',
                                        title="all_models, cov_mean, COMMON INSTANCES",)

# avg_scores_small = plot_average_scores(dataset_scores, model_scores, color_mapping, small_models)

# plt.plot(avg_scores['n_samples'], avg_scores['score'], color='black', label='all models')
# plt.plot(avg_scores_small['n_samples'], avg_scores_small['score'], color='blue', label='small models')

In [ ]:
avg_scores_large = plot_average_scores(dataset_scores, model_scores, 
                                       color_mapping, large_models,
                                       log_axis=LOG_AXIS,
                                       x_key='coverage',
                                       title=f"Large models, Coverage, NGRAM={N_GRAMS}")

avg_scores_small = plot_average_scores(dataset_scores, model_scores,
                                       color_mapping, small_models,
                                       log_axis=LOG_AXIS,
                                       x_key='coverage',
                                       title=f"Small models, Coverage, NGRAM={N_GRAMS}")

avg_scores = plot_average_scores(dataset_scores, model_scores,
                                 color_mapping, models,
                                 log_axis=LOG_AXIS,
                                 x_key='coverage',                                 title=f"All models, Coverage, NGRAM={N_GRAMS}")

In [ ]:
import plotly.graph_objs as go
import collections
import numpy as np


# Modified function to create an interactive plot using Plotly within a Jupyter Notebook
def plot_average_scores_plotly(dataset_scores, model_scores, color_mapping, 
                              model_list=None, x_key='n_samples',
                              log_axis=False, title=None):
    avg_dataset_scores = collections.defaultdict(lambda: collections.defaultdict(list))
    avg_scores = {'score': [], x_key: []} 

    # Create Plotly figure
    fig = go.Figure()

    for dataset in dataset_scores:
        if model_list is None:
            avg_dataset_scores[dataset]['score'] = np.mean(dataset_scores[dataset]['score'])
            avg_dataset_scores[dataset][x_key] = np.mean(dataset_scores[dataset][x_key])
        else:
            model_indices = [i for i, model in enumerate(model_scores.keys()) if model in model_list]
            avg_dataset_scores[dataset]['score'] = np.mean([dataset_scores[dataset]['score'][i] for i in model_indices])
            avg_dataset_scores[dataset][x_key] = np.mean([dataset_scores[dataset][x_key][i] for i in model_indices])

        # Append data for plotting
        avg_scores['score'].append(avg_dataset_scores[dataset]['score'])
        avg_scores[x_key].append(avg_dataset_scores[dataset][x_key])
        
        # Add trace for each dataset to the figure
        fig.add_trace(go.Scatter(
            x=[avg_dataset_scores[dataset][x_key]],
            y=[avg_dataset_scores[dataset]['score']],
            text=[dataset],  # Will show this text on hover
            mode='markers',
            marker=dict(color=color_mapping[dataset]),
            name=dataset
        ))

    # sort by x_key (e.g., 'n_samples')
    sorted_indices = np.argsort(avg_scores[x_key])
    sorted_scores = np.array(avg_scores['score'])[sorted_indices]
    sorted_x_values = np.array(avg_scores[x_key])[sorted_indices]

    # Add sorted average line to the figure
    fig.add_trace(go.Scatter(
        x=sorted_x_values,
        y=sorted_scores,
        mode='lines',
        marker=dict(color='red'),
        name='Average'
    ))

    # Set log axis if specified
    if log_axis:
        fig.update_xaxes(type='log')

    # Update layout with title and axis labels
    fig.update_layout(
        title=title,
        xaxis_title=x_key,
        yaxis_title='Score',
        hovermode='closest'
    )

    # disable legend
    fig.update_layout(showlegend=False)

    # Show figure
    fig.show()

    return avg_scores

avg_scores = plot_average_scores_plotly(dataset_scores, 
                                       model_scores, 
                                       color_mapping,
                                       model_list=large_models,
                                       log_axis=LOG_AXIS,
                                       x_key='cov_mean',)

In [ ]:
# Assuming your dictionary is named `dataset_scores`
sorted_data = sorted(dataset_scores.items(), key=lambda x: max(x[1]['n_samples']), reverse=True)
top_5_datasets = {k: v['n_samples'] for k, v in sorted_data[:5]}
print(f"Top 5 datasets: {top_5_datasets}")

In [ ]:
import matplotlib.pyplot as plt
from adjustText import adjust_text


# wa.plot_scores(model_scores, dataset_scores, model_color_mapping, name="XY_pairs")
wa.plot_scores(model_scores, dataset_scores, color_mapping,
               model_color_mapping, name="XY_pairs",
               log_axis=True)

In [ ]:
# wa.plot_scores(single_model_score, single_model_dataset_score, color_mapping,
#                model_color_mapping, name="XY_pairs",
#                log_axis=True, x_key='coverage')

In [ ]:
wa.plot_scores(model_scores, dataset_scores, color_mapping,
            model_color_mapping, name="XY_pairs",
            x_key='coverage_', log_axis=True)


In [ ]:
['pythia-12b', 'pythia-6.9b', 'pythia-2.8b', 'pythia-1.4b', 'pythia-410m', 'pythia-160m', 'pythia-70m', 'pythia-31m', 'pythia-14m']

model_param_map = {'pythia-12b': 12e09,
                    'pythia-6.9b': 6.9e09,
                    'pythia-2.8b': 2.8e09,
                    'pythia-1.4b': 1.4e09,
                    'pythia-410m': 410e06,
                    'pythia-160m': 160e06,
                    'pythia-70m': 70e06,
                    'pythia-31m': 31e06,
                    'pythia-14m': 14e06,}

# wa.plot_model_size_vs_scores(results_dict, models, model_param_map, color_mapping)


In [ ]:
# wa.analyze_and_plot_distributions(task_dfs_all)

In [ ]:
# get total samples per language
task_dfs_all_total_samples = {}
for lang, df in task_dfs_all.items():
    task_dfs_all_total_samples[lang] = df['value'].sum()

model_scores_all, dataset_scores_all = wa.prepare_scores(results_dict, task_dfs_all, models,
                                                         coverage_=task_cov_all_mean,
                                                         cov_mean=task_cov_all_mean)

In [ ]:
# n_gram_freqs_all['ja-en']['value'] = pd.to_numeric(n_gram_freqs_all['ja-en']['value'], errors='coerce')
# largest_values = n_gram_freqs_all['ja-en'].nlargest(50, columns=['value'])

# print(largest_values)


In [ ]:
wa.plot_scores(model_scores_all, dataset_scores_all, 
            color_mapping, model_color_mapping, 
            name=f"all instances NGRAM={N_GRAMS}",
            log_axis=LOG_AXIS)

wa.plot_scores(model_scores_all, dataset_scores_all, 
            color_mapping, model_color_mapping, 
            name=f"all instances NGRAM={N_GRAMS}",
            x_key="coverage",
            log_axis=LOG_AXIS)


In [ ]:
wa.plot_scores(model_scores_all, dataset_scores_all, 
            color_mapping, model_color_mapping, 
            name=f"ALL INSTANCES NGRAM={N_GRAMS}",
            x_key="coverage_",
            log_axis=LOG_AXIS)

In [ ]:
avg_scores_large_all = plot_average_scores(dataset_scores_all, model_scores_all, 
                                          color_mapping, large_models,
                                          x_key='coverage_', log_axis=False,
                                          title=f"Large models, NGRAM={N_GRAMS}, ALL INSTANCES")

In [ ]:
avg_scores_large_all = plot_average_scores(dataset_scores_all, model_scores_all, 
                                          color_mapping, large_models,
                                          x_key='coverage_', log_axis=False,
                                          title=f"Large models, Coverage_, NGRAM={N_GRAMS}, ALL INSTANCES")
avg_scores_small_all = plot_average_scores(dataset_scores_all, model_scores_all,
                                          color_mapping, small_models,
                                          x_key='coverage_', log_axis=False,
                                          title=f"Small models, Coverage_, NGRAM={N_GRAMS}, ALL INSTANCES")
avg_scores_all = plot_average_scores(dataset_scores_all, model_scores_all,
                                    color_mapping, models,
                                    x_key='coverage_', log_axis=False,
                                    title=f"All models, Coverage_, NGRAM={N_GRAMS}, ALL INSTANCES")

In [ ]:
avg_scores_large_all = plot_average_scores(dataset_scores_all, model_scores_all, 
                                          color_mapping, large_models,
                                          x_key='n_samples', log_axis=True,
                                          title=f"Large models, NGRAM={N_GRAMS}, ALL INSTANCES")
avg_scores_small_all = plot_average_scores(dataset_scores_all, model_scores_all,
                                          color_mapping, small_models,
                                          x_key='n_samples', log_axis=True,
                                          title=f"Small models, NGRAM={N_GRAMS}, ALL INSTANCES")
avg_scores_all = plot_average_scores(dataset_scores_all, model_scores_all,
                                    color_mapping, models,
                                    x_key='n_samples', log_axis=True,
                                    title=f"All models, NGRAM={N_GRAMS}, ALL INSTANCES")

In [ ]:
avg_scores_large_all = plot_average_scores(dataset_scores_all, model_scores_all, 
                                          color_mapping, large_models,
                                          x_key='coverage', log_axis=True,
                                          title=f"Large models, Coverage, NGRAM={N_GRAMS}, ALL INSTANCES")
avg_scores_small_all = plot_average_scores(dataset_scores_all, model_scores_all,
                                          color_mapping, small_models,
                                          x_key='coverage', log_axis=True,
                                          title=f"Small models, Coverage, NGRAM={N_GRAMS}, ALL INSTANCES")
avg_scores_all = plot_average_scores(dataset_scores_all, model_scores_all,
                                    color_mapping, models,
                                    x_key='coverage', log_axis=True,
                                    title=f"All models, Coverage, NGRAM={N_GRAMS}, ALL INSTANCES")

In [ ]:
plot_key = 'coverage'
log_axis = True
avg_scores_large_all = plot_average_scores_plotly(dataset_scores_all, model_scores_all, 
                                          color_mapping, large_models,
                                          x_key=plot_key, log_axis=log_axis,
                                          title=f"large models, NGRAM={N_GRAMS}, ALL INSTANCES")
avg_scores_small_all = plot_average_scores_plotly(dataset_scores_all, model_scores_all,
                                          color_mapping, small_models,
                                          x_key=plot_key, log_axis=log_axis,
                                          title=f"small models, NGRAM={N_GRAMS}, ALL INSTANCES")
avg_scores_all = plot_average_scores_plotly(dataset_scores_all, model_scores_all,
                                    color_mapping, None,
                                    x_key=plot_key, log_axis=log_axis,
                                    title=f"all models, NGRAM={N_GRAMS}, ALL INSTANCES")

In [ ]:
# wa.plot_scores(model_scores, dataset_scores, 
#             color_mapping, model_color_mapping, 
#             name="XY_pairs", log_axis=LOG_AXIS,
#             x_key="coverage")

In [ ]:
# for example in task_dfs_all['world_religions']['example']:
#     print(example['question'])
#     break

# examples = list(task_dfs_all['world_religions']['example'])
# examples[5]['question']


In [84]:
# Initialize a dictionary to hold the contents of results.json for each model/dataset pair
results_dict = collections.defaultdict(dict)
instance_results_dict = collections.defaultdict(dict)

# Iterate over each model and dataset, loading the results.json file

for base_results_path, models_ in base_results_paths.items():
    for model in models_:
        results_path = os.path.join(base_results_path, model, TASK, SHOT, 'results.json')
        instance_results_path = os.path.join(base_results_path, model, TASK, SHOT, 'doc_results.json')
        results = json.load(open(results_path, 'r'))['results']
        doc_results = json.load(open(instance_results_path, 'r'))
        for task in results.keys():
            # remove hendrycksTest-
            task_str = task.split("-")[1]
            if task_str in TASKS:
                results_dict[task_str][model] = results[task]['acc']
                if os.path.exists(instance_results_path):
                    instance_results_dict[model][task_str] = doc_results[task]

In [86]:
# instance_results_dict['pythia-12b']['world_religions'][0]

In [87]:
def process_instance_results(instance_results, task_dfs, label=''):
    model_instance_results = collections.defaultdict(list)
    for model in instance_results.keys():
        for i, task in enumerate(instance_results[model].keys()):
            model_task_results = instance_results[model][task].copy()
            # Add task label to each id, with an optional common label
            for row in model_task_results:
                row['id'] = f"{str(row['id'])}_{i}"
            model_instance_results[model].extend(model_task_results)
    return model_instance_results

def merge_and_process_dfs(task_dfs):
    example_dfs = pd.concat(task_dfs.values())
    # Extract 'query' from the 'example' column
    example_dfs['query'] = example_dfs['example'].apply(lambda x: x['question'])
    # Perform the aggregation to get the sum of 'value' and other statistics if needed
    aggregated_data = example_dfs.groupby(['query', 'task'])['value'].agg(['sum', 'count']).reset_index()
    # Merge the aggregated data back to the original DataFrame
    example_dfs = example_dfs.merge(aggregated_data, on=['query', 'task'], how='left')
    # Sort the DataFrame based on the 'sum' of 'value'
    example_dfs = example_dfs.sort_values(by='sum', ascending=False)
    # rename "pair" column to query
    # example_dfs = example_dfs.rename(columns={'pair': 'query'})
    
    # example_dfs = example_dfs.groupby(['query', 'task'])['value'].sum().sort_values(ascending=False).reset_index()
    return example_dfs

# Process all tasks
model_instance_results_all = process_instance_results(instance_results_dict.copy(), task_dfs_all.copy(), label='_all')
example_dfs_all = merge_and_process_dfs(task_dfs_all)

# Process common tasks
model_instance_results_common = process_instance_results(instance_results_dict.copy(), task_dfs_common.copy(), label='_common')
example_dfs_common = merge_and_process_dfs(task_dfs_common)

In [88]:
# from rapidfuzz import process, fuzz

# # Create a new column for the id
# example_dfs_all['id'] = None
# instances = pd.DataFrame(model_instance_results['pythia-12b'])

# for instance in tqdm(instances):
#     # Skip if this instance has already been matched
#     # if example_dfs_all.loc[example_dfs_all['query'] == instance['query'], 'id'].notna().any():
#         # continue

#     # Get the best match in example_dfs_all['query']
#     best_match, score, _ = process.extractOne(instance['query'], example_dfs_all['query'], scorer=fuzz.token_sort_ratio)
    
#     # If the score is above a certain threshold, assign the id
#     example_dfs_all.loc[example_dfs_all['query'] == best_match, 'id'] = instance['id']

# # Save the DataFrame to a CSV file in BASE_PATH
# example_dfs_all.to_csv(os.path.join(BASE_PATH, 'example_dfs_all_.csv'), index=False)

# # instances = model_instance_results['pythia-12b']
# # for row, example in enumerate(tqdm(example_dfs_all['query'])):
# #     for instance in instances:
# #         if example == instance['query']:
# #             print(f"Found match for {example}")
# #             example_dfs_all.loc[row, 'id'] = instance['id']
# #             instances.remove(instance)
# #             break

In [89]:
# import pandas as pd
# import os
# from tqdm import tqdm

# def normalize_text(text):
#     # Convert to lowercase, strip whitespace, remove punctuation, etc.
#     return text.lower().strip()

# def match_and_update_ids(example_dfs, model_instance_results, filename, key='query'):
#     # Convert model_instance_results to a DataFrame
#     instances = pd.DataFrame(model_instance_results)

#     # Normalize the 'query' column in both DataFrames
#     example_dfs[key] = example_dfs[key].apply(normalize_text)
#     instances[key] = instances[key].apply(normalize_text)

#     print(f"example_dfs: {example_dfs.head(1)}")
#     print(f"instances: {instances.head(1)}")
#     # Merge the example_dfs with instances on 'query' column
#     merged_df = example_dfs.merge(instances[['id', key]], on=key, how='left')

#     # Save the merged DataFrame to a CSV file if it doesn't exist
#     if not os.path.exists(filename):
#         merged_df.to_csv(filename, index=False)
    
#     return merged_df

# # Assuming BASE_PATH is defined
# # Process all tasks
# example_dfs_all_filename = os.path.join(BASE_PATH, 'example_dfs_all_exact.csv')
# example_dfs_all = match_and_update_ids(example_dfs_all, model_instance_results_all['pythia-12b'], example_dfs_all_filename)

# # Process common tasks
# example_dfs_common_filename = os.path.join(BASE_PATH, 'example_dfs_common_exact.csv')
# example_dfs_common = match_and_update_ids(example_dfs_common, model_instance_results_common['pythia-12b'], example_dfs_common_filename)

In [90]:
# Enable tqdm for pandas apply
tqdm.pandas()

def find_matching_id(row, instances_df):
    # Use regex=False to avoid interpreting the pattern as a regular expression
    match = instances_df[instances_df['query'].str.contains(row['query'], na=False, regex=False)]
    if not match.empty:
        return match.iloc[0]['id']
    return None

def add_ids_and_save(example_dfs, instances, filename):
    # Add a new column 'id' to example_dfs by applying the find_matching_id function with a progress bar
    example_dfs['id'] = example_dfs.progress_apply(find_matching_id, axis=1, instances_df=instances)

    # Save the DataFrame to CSV
    example_dfs.to_csv(filename, index=False)

    return example_dfs

# Assuming model_instance_results is a list of dictionaries
instances_all_df = pd.DataFrame(model_instance_results_all['pythia-12b'])
instances_common_df = pd.DataFrame(model_instance_results_common['pythia-12b'])

# Assuming BASE_PATH is defined
example_dfs_all_filename = os.path.join(BASE_PATH, 'example_dfs_all_exact.csv')
example_dfs_common_filename = os.path.join(BASE_PATH, 'example_dfs_common_exact.csv')

# Process all tasks
# example_dfs_all = add_ids_and_save(example_dfs_all, instances_all_df, example_dfs_all_filename)

# Process common tasks
example_dfs_common = add_ids_and_save(example_dfs_common, instances_common_df, example_dfs_common_filename)

  1%|          | 4973/450981 [00:09<13:43, 541.68it/s]


KeyboardInterrupt: 

In [ ]:
# load it
# example_dfs_all_0 = pd.read_csv(os.path.join(BASE_PATH, "_0", "example_dfs_all_exact_0.csv"))
# example_dfs_common_0 = pd.read_csv(os.path.join(BASE_PATH, "_0", "example_dfs_common_exact_0.csv"))

In [ ]:
# # load it
# examples_dfs_all = pd.read_csv(os.path.join(BASE_PATH, 'example_dfs_all_exact.csv'))
# examples_dfs_common = pd.read_csv(os.path.join(BASE_PATH, 'example_dfs_common_exact.csv'))

In [ ]:
import numpy as np
import pandas as pd
import os
import pickle
from tqdm import tqdm

# Define the softmax function
def softmax(x):
    e_x = np.exp(x - np.max(x))  # subtract max(x) for numerical stability
    return e_x / e_x.sum()

def nll_to_prob(nll):
    return np.exp(nll)

def process_and_save_results(example_dfs, model_instance_results, base_path, suffix):
    examples_dfs_models = {}
    for model in model_instance_results.keys():
        instances = pd.DataFrame(model_instance_results[model])
        examples_dfs_model = example_dfs.merge(instances, on='id', how='inner')
        examples_dfs_model['probs_softmax'] = examples_dfs_model['lls'].apply(softmax)
        examples_dfs_model['probs_nll'] = examples_dfs_model['lls'].apply(nll_to_prob)
        examples_dfs_model['probs_softmax_gold'] = examples_dfs_model.apply(lambda row: row['probs_softmax'][row['gold']], axis=1)
        examples_dfs_model['probs_nll_gold'] = examples_dfs_model.apply(lambda row: row['probs_nll'][row['gold']], axis=1)
        examples_dfs_models[model] = examples_dfs_model

    # Save model_instance_results to pickle
    with open(os.path.join(base_path, f'model_instance_results_{suffix}.pkl'), 'wb') as f:
        pickle.dump(model_instance_results, f)

    # Save examples_dfs_models to pickle
    with open(os.path.join(base_path, f'examples_dfs_{suffix}_models.pkl'), 'wb') as f:
        pickle.dump(examples_dfs_models, f)
    
    # # save example_dfs to pkl
    # with open(os.path.join(base_path, f'example_dfs_{suffix}.pkl'), 'wb') as f:
    #     pickle.dump(example_dfs, f)

    return examples_dfs_models
    
    

# Assuming BASE_PATH is defined
# Process all tasks
# example_dfs_all_models = process_and_save_results(example_dfs_all, model_instance_results_all, BASE_PATH, 'all')

# Process common tasks
example_dfs_common_models = process_and_save_results(example_dfs_common, model_instance_results_common, BASE_PATH, 'common')

In [ ]:
# count how many examples are not None or Nan
examples_matched = example_dfs_all['id'].notna().sum()
examples_umatched = example_dfs_all['id'].isna().sum()

print(f"Examples matched: {examples_matched}")
print(f"Examples unmatched: {examples_umatched}")